# Multiarmed bandit

Il problema del **multi-armed bandit** è un caso particolare del problema generale del Reinforcement Learning.
In esso è presente un solo stato, in cui, ad ogni step, l'agente può scegliere una di $n$ azioni, ciascuna delle quali restituisce un reward da una
diversa distribuzione di probabilità. Lo scopo dell'agente è, come al solito, quello di **massimizzare il ritorno totale**.
Nel caso del multi-armed bandit è centrale il **dilemma exploration vs exploitation**, poiché l'agente deve sfruttare la sua conoscenza provvisoria dei valori delle azioni per cercare di massimizzare il ritorno (exploitation), ma allo stesso tempo deve continuare ad aggiornarli (exploration).
<br><br>
Il problema del multi-armed bandit può essere esemplificato da un giocatore d'azzardo che deve cercare di capire quale, di n slot-machine, è più remunerativa. Il problema prende il nome proprio dalle vecchie slot machine, che venivano chiamate, per motivi comprensibili, **one-armed bandit**.

<img src="img/slot_machines.JPG" alt="slot machines" style="width: 400px;"/>
<center>source: Wikipedia</center>

In [ ]:
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class BernoulliMultiarmedBandit:
    ''' Rappresenta un multiarmed bandit in cui il braccio i-esimo restituisce
    un reward 1 con probabilità p_i e 0 con probabilità 1 - p_i

    Attributi
    ---------
    ps : una lista con i valori di p per ogni braccio'''

    def __init__(self, ps):
        self.ps = ps

    def get_arms_number(self):
        ''' Ritorna il numero di braccia del Bandit'''
        return len(self.ps)

    def pull_arm(self, idx):
        ''' Tira un braccio del Bandit

        Argomenti
        ---------
        idx : indice del braccio da tirare

        Ritorno
        -------
        Un reward: 1 o 0'''
        return random.choices([1, 0], weights=[self.ps[idx], 1 - self.ps[idx]], k=1)[0]

# Epsilon-greedy

Ad ogni step l'agente sceglie l'azione con il valore stimato più alto con probabilità $\epsilon + \frac{\epsilon}{\left|A\right|}$ (exploitation), un'altra azione con probabilità $1 - \epsilon - \frac{\epsilon}{\left|A\right|}$ (exploration), dove $\left|A\right|$ è il numero delle azioni.

In [ ]:
class EpsilonGreedyPlayer:
    ''' Giocatore che utilizza la strategia epsilon-greedy

    Attributi
    ---------
    epsilon : il fattore di esplorazione'''

    def __init__(self, bandit, epsilon, init_value=0):
        '''
        Argomenti
        ---------
        init_values : valori iniziali delle azioni'''

        self.bandit = bandit
        self.epsilon = epsilon
        self.init_values = [init_value] * bandit.get_arms_number()

    def _reset(self):
        self.actions_means = list(self.init_values)
        self.actions_counts = [0] * self.bandit.get_arms_number()

    def _get_random_action(self):
        ''' Sceglie un'azione a caso'''
        return random.randrange(self.bandit.get_arms_number())

    def _get_optimal_action(self):
        max_value = max(self.actions_means)
        # ritorna l'indice della prima tra le azioni che hanno valore massimo
        return self.actions_means.index(max_value)

    def _update_action_mean_reward(self, action, reward):
        ''' Aggiorna il valore di un'azione

        Argomenti
        ---------
        action : indice dell'azione da aggiornare
        reward : nuovo reward ottenuto per l'azione
        '''
        if self.actions_counts[action] == 0:
            self.actions_means[action] = reward
        else:
            self.actions_means[action] = \
                (self.actions_means[action]*self.actions_counts[action] + reward) /\
                (self.actions_counts[action] + 1)

        # aggiorna il conteggio dell'azione
        self.actions_counts[action] += 1

    def _play_one_action(self):
        ''' Sceglie un'azione secondo la strategia implementata.
        
        Ritorno
        -------
        action : l'azione scelta
        reward : il reward ottenuto'''
        if random.random() < self.epsilon:
            # sceglie un'azione a caso (exploration)
            action = self._get_random_action()
        else:
            # sceglie una delle azioni migliori (exploitation)
            action = self._get_optimal_action()

        # effettua l'azione scelta
        reward = self.bandit.pull_arm(action)

        # aggiorna il reward medio dell'azione scelta
        self._update_action_mean_reward(action, reward)

        return action, reward

    def _play_one_game(self, game_length):
        ''' Gioca una partita seguendo la strategia implementata
        
        Parametri
        ---------
        game_length : la lunghezza del gioco
        
        Ritorno
        -------
        actions : una lista con le azioni scelte
        rewards : una lista con i reward ottenuti'''
        self._reset()
        actions, rewards = [], []
        for _ in range(game_length):
            action, reward = self._play_one_action()
            actions.append(action)
            rewards.append(reward)
        return actions, rewards

In [ ]:
# numero di partite da giocare per ottenere le statistiche
NUM_GAMES = 1_000
# lunghezza di ciascuna partita
GAME_LENGTH = 200

bandit = BernoulliMultiarmedBandit(
    ps=[0.2, 0.1, 0.8, 0.6, 0.2, 0.5, 0.3, 0.7, 0.9])


stats = defaultdict(lambda: dict(actions=[], rewards=[]))
for epsilon in  [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    player = EpsilonGreedyPlayer(bandit, epsilon=epsilon)
    for i in range(NUM_GAMES):
        actions, rewards = player._play_one_game(GAME_LENGTH)
        stats[epsilon]['actions'].append(actions)
        stats[epsilon]['rewards'].append(rewards)


# mostra i reward cumulativi medi
fig, ax = plt.subplots(nrows=1, ncols=1)
epsilons = []
mean_total_rewards = []
for epsilon in stats.keys():
    epsilons.append(epsilon)
    all_episodes_rewards = stats[epsilon]['rewards']
    all_total_rewards = [sum(one_game_rewards)
                         for one_game_rewards in all_episodes_rewards]
    mean_total_rewards.append(np.mean(all_total_rewards))

ax.plot(epsilons, mean_total_rewards)

# mostra i reward medi
fig, ax = plt.subplots(nrows=1, ncols=1)
for epsilon in stats.keys():
    all_episodes_rewards = stats[epsilon]['rewards']
    mean_rewards = np.mean(np.stack(all_episodes_rewards), axis=0)
    ax.plot(mean_rewards, label=f'epsilon={epsilon}', linewidth=0.5, alpha=1.)
    ax.legend()

# Softmax

Adottando la strategia $\epsilon$-greedy l'agente esplora con la stessa frequenza azioni con diversi valori stimati e quindi con diversa probabilità di essere ottimali. La strategia **Softmax** risolve questo problema favorendo le azioni più promettenti.
<br>
La probabilità che l'azione $a_j$ venga scelta è
$$P(a_j) = \frac{e^{\frac{R(a_j)}{\tau}}}{\sum_{i=1}^{\left|A\right|}e^\frac{R(a_i)}{\tau}}$$
<br>
$R(a_i)$ è il **valore stimato** dell'azione $i$-esima.
<br>
Il parametro $\tau$ è chiamato **temperatura**. All'aumentare di $\tau$ la distribuzione di probabilità delle diverse azioni tende alla **distribuzione uniforme**.

In [ ]:
class SoftmaxPlayer:
    ''' Giocatore che utilizza la strategia Softmax

    Attributi
    ---------
    tau : la temperatura. Al suo aumentare aumenta l'esplorazione e viceversa'''

    def __init__(self, bandit, tau, init_value=0):
        '''
        Argomenti
        ---------
        init_values : valori iniziali delle azioni'''

        self.bandit = bandit
        self.tau = tau
        self.init_values = [init_value] * bandit.get_arms_number()

    def _reset(self):
        self.actions_means = list(self.init_values)
        self.actions_counts = [0] * self.bandit.get_arms_number()

    def _update_action_mean_reward(self, action, reward):
        ''' Aggiorna il valore di un'azione

        Argomenti
        ---------
        action : indice dell'azione da aggiornare
        reward : nuovo reward ottenuto per l'azione
        '''
        if self.actions_counts[action] == 0:
            self.actions_means[action] = reward
        else:
            self.actions_means[action] = \
                (self.actions_means[action]*self.actions_counts[action] + reward) /\
                (self.actions_counts[action] + 1)

        # aggiorna il conteggio dell'azione
        self.actions_counts[action] += 1

    def _softmax(self, nums, tau):
        nums = np.asarray(nums)
        return np.exp(nums / tau).tolist()

    def _play_one_action(self):
        ''' Sceglie un'azione secondo la strategia implementata.
        
        Ritorno
        -------
        action : l'azione scelta
        reward : il reward ottenuto'''
        weights = self._softmax(self.actions_means, self.tau)
        # sceglie una delle azioni pesandole con con i valori restituiti dalla softmax
        action = random.choices(
            range(self.bandit.get_arms_number()), weights=weights, k=1)[0]
        reward = self.bandit.pull_arm(action)
        # aggiorna il reward medio dell'azione scelta
        self._update_action_mean_reward(action, reward)
        return action, reward

    def _play_one_game(self, game_length):
        ''' Gioca una partita seguendo la strategia implementata
        
        Parametri
        ---------
        game_length : la lunghezza del gioco
        
        Ritorno
        -------
        actions : una lista con le azioni scelte
        rewards : una lista con i reward ottenuti'''
        self._reset()
        actions, rewards = [], []
        for _ in range(game_length):
            action, reward = self._play_one_action()
            actions.append(action)
            rewards.append(reward)
        return actions, rewards

In [ ]:
# numero di partite da giocare per ottenere le statistiche
NUM_GAMES = 1_000
# lunghezza di ciascuna partita
GAME_LENGTH = 200

bandit = BernoulliMultiarmedBandit(
    ps=[0.2, 0.1, 0.8, 0.6, 0.2, 0.5, 0.3, 0.7, 0.9])


stats = defaultdict(lambda: dict(actions=[], rewards=[]))
for tau in [0.05, 0.08, 0.1, 0.12, 0.15, 0.20]:
    player = SoftmaxPlayer(bandit, tau=tau)
    for i in range(NUM_GAMES):
        actions, rewards = player._play_one_game(GAME_LENGTH)
        stats[tau]['actions'].append(actions)
        stats[tau]['rewards'].append(rewards)


# mostra i reward cumulativi medi
fig, ax = plt.subplots(nrows=1, ncols=1)
taus = []
mean_total_rewards = []
for tau in stats.keys():
    taus.append(tau)
    all_episodes_rewards = stats[tau]['rewards']
    all_total_rewards = [sum(one_game_rewards)
                         for one_game_rewards in all_episodes_rewards]
    mean_total_rewards.append(np.mean(all_total_rewards))

ax.plot(taus, mean_total_rewards)

# mostra i reward medi
fig, ax = plt.subplots(nrows=1, ncols=1)
for tau in stats.keys():
    all_episodes_rewards = stats[tau]['rewards']
    mean_rewards = np.mean(np.stack(all_episodes_rewards), axis=0)
    ax.plot(mean_rewards, label=f'tau={tau}', linewidth=0.5, alpha=1.)
    ax.legend()

# Thompson sampling

Né la strategia $epsilon$-greedy né quella Softmax tengono conto dell'**incertezza** della stima dei valori delle azioni, mentre è ragionevole pensare che - date, ad esempio, due azioni aventi lo stesso valore stimato - l'agente debba favorire l'esplorazione di quella il cui valore è stimato con maggiore incertezza.
Con la strategia del Thompson sampling l'agente costruisce e aggiorna continuamente la **distribuzione di probabilità** del valore di ogni azione, ad ogni step estrae un valore da ogni distribuzione ed effettua l'azione corrispondente al valore estratto più alto.
<br> 

In [ ]:
class ThompsonPlayer:
    ''' Giocatore che utilizza il Thompson sampling'''

    def __init__(self, bandit):
        self.bandit = bandit

    def _reset(self):
        self.alphas = [1] * self.bandit.get_arms_number()
        self.betas = [1] * self.bandit.get_arms_number()

    def _update_alphas_betas(self, action, reward):
        ''' Aggiorna i valori di alpha e beta per un'azione

        Argomenti
        ---------
        action : indice dell'azione da aggiornare
        reward : nuovo reward ottenuto per l'azione
        '''
        if reward == 1:
            self.alphas[action] += 1
        else:
            self.betas[action] += 1

    def _play_one_action(self):
        ''' Sceglie un'azione secondo la strategia implementata.
        
        Ritorno
        -------
        action : l'azione scelta
        reward : il reward ottenuto'''
        # estrae dei valori a caso dalle distribuzioni beta associate alle braccia
        values = [np.random.beta(alpha, beta) for alpha, beta in zip(self.alphas, self.betas)]
        # sceglie l'azione col valore estratto maggiore
        action = values.index(max(values))
        reward = self.bandit.pull_arm(action)
        # aggiorna il reward medio dell'azione scelta
        self._update_alphas_betas(action, reward)
        return action, reward

    def _play_one_game(self, game_length):
        ''' Gioca una partita seguendo la strategia implementata
        
        Parametri
        ---------
        game_length : la lunghezza del gioco
        
        Ritorno
        -------
        actions : una lista con le azioni scelte
        rewards : una lista con i reward ottenuti'''
        self._reset()
        actions, rewards = [], []
        for _ in range(game_length):
            action, reward = self._play_one_action()
            actions.append(action)
            rewards.append(reward)
        return actions, rewards

In [ ]:
# numero di partite da giocare per ottenere le statistiche
NUM_GAMES = 1_000
# lunghezza di ciascuna partita
GAME_LENGTH = 200

bandit = BernoulliMultiarmedBandit(
    ps=[0.2, 0.1, 0.8, 0.6, 0.2, 0.5, 0.3, 0.7, 0.9])


stats = dict(actions=[], rewards=[])
player = ThompsonPlayer(bandit)
for i in range(NUM_GAMES):
    actions, rewards = player._play_one_game(GAME_LENGTH)
    stats['actions'].append(actions)
    stats['rewards'].append(rewards)


# mostra i reward cumulativo medio
all_episodes_rewards = stats['rewards']
all_total_rewards = [sum(one_game_rewards)
                    for one_game_rewards in all_episodes_rewards]
print(np.mean(all_total_rewards))

# mostra i reward medi
fig, ax = plt.subplots(nrows=1, ncols=1)
all_episodes_rewards = stats['rewards']
mean_rewards = np.mean(np.stack(all_episodes_rewards), axis=0)
ax.plot(mean_rewards, linewidth=0.5, alpha=1.)